In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import fuzz, process
import re
import string
from difflib import SequenceMatcher
import os

# TASK 1 - MATCH USER QUERIES WITH RESOLVED QUERIES

In [ ]:
def preprocess_text(text):
    """
    Preprocess text for better matching by:
    - Converting to lowercase
    - Removing punctuation
    - Removing extra whitespace
    - Removing common stop words if needed
    """
    if pd.isna(text):
        return ""
    
    # Convert to lowercase
    text = str(text).lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove extra whitespace
    text = ' '.join(text.split())
    
    return text

def load_query_data():
    """
    Load resolved and new queries from files
    Files structure:
    - resolved_queries.csv: Query_ID, Pre_Resolved_Query
    - new_queries.csv: Variation_Query, Matches_With_Query_ID
    """
    try:
        # Load resolved queries
        resolved_df = pd.read_csv('text_search_data/resolved_queries.csv')
        print("Resolved queries columns:", resolved_df.columns.tolist())
        print("Sample resolved queries:")
        print(resolved_df.head())
        
        # Load new queries  
        new_df = pd.read_csv('text_search_data/new_queries.csv')
        print("\nNew queries columns:", new_df.columns.tolist())
        print("Sample new queries:")
        print(new_df.head())
        
        # Rename columns for consistency with rest of the code
        resolved_df = resolved_df.rename(columns={
            'Query_ID': 'id',
            'Pre_Resolved_Query': 'query'
        })
        
        new_df = new_df.rename(columns={
            'Variation_Query': 'query',
            'Matches_With_Query_ID': 'expected_match_id'
        })
        
        # Add sequential IDs for new queries if not present
        if 'id' not in new_df.columns:
            new_df['id'] = range(1, len(new_df) + 1)
        
        return resolved_df, new_df
        
    except FileNotFoundError as e:
        print(f"Files not found: {e}")
        print("Please ensure files are in the text_search_data folder with correct names:")
        print("- resolved_queries.csv")
        print("- new_queries.csv")

def fuzzy_search_matching(resolved_queries, new_queries, threshold=80):
    """
    Match new queries with resolved queries using FuzzyWuzzy
    Tests different fuzzy search methods to find the best one
    """
    results = []
    
    # Preprocess queries
    resolved_processed = [preprocess_text(q) for q in resolved_queries['query']]
    new_processed = [preprocess_text(q) for q in new_queries['query']]
    
    for i, new_query in enumerate(new_processed):
        best_matches = {
            'ratio': None,
            'partial_ratio': None,
            'token_sort_ratio': None,
            'token_set_ratio': None
        }
        
        # Test different fuzzy methods
        for j, resolved_query in enumerate(resolved_processed):
            # Simple ratio
            ratio_score = fuzz.ratio(new_query, resolved_query)
            if best_matches['ratio'] is None or ratio_score > best_matches['ratio'][1]:
                best_matches['ratio'] = (j, ratio_score)
            
            # Partial ratio (good for substring matches)
            partial_score = fuzz.partial_ratio(new_query, resolved_query)
            if best_matches['partial_ratio'] is None or partial_score > best_matches['partial_ratio'][1]:
                best_matches['partial_ratio'] = (j, partial_score)
            
            # Token sort ratio (good for word order differences)
            token_sort_score = fuzz.token_sort_ratio(new_query, resolved_query)
            if best_matches['token_sort_ratio'] is None or token_sort_score > best_matches['token_sort_ratio'][1]:
                best_matches['token_sort_ratio'] = (j, token_sort_score)
            
            # Token set ratio (good for different word sets)
            token_set_score = fuzz.token_set_ratio(new_query, resolved_query)
            if best_matches['token_set_ratio'] is None or token_set_score > best_matches['token_set_ratio'][1]:
                best_matches['token_set_ratio'] = (j, token_set_score)
        
        # Find the best overall method for this query
        best_method = max(best_matches.keys(), key=lambda x: best_matches[x][1])
        best_idx, best_score = best_matches[best_method]
        
        # Check if we have expected match for comparison
        expected_match_id = new_queries.iloc[i].get('expected_match_id', None)
        is_correct_match = (resolved_queries.iloc[best_idx]['id'] == expected_match_id) if expected_match_id else "Unknown"
        
        if best_score >= threshold:
            results.append({
                'new_query_id': new_queries.iloc[i]['id'],
                'new_query': new_queries.iloc[i]['query'],
                'matched_resolved_id': resolved_queries.iloc[best_idx]['id'],
                'matched_resolved_query': resolved_queries.iloc[best_idx]['query'],
                'expected_match_id': expected_match_id,
                'is_correct_match': is_correct_match,
                'similarity_score': best_score,
                'best_method': best_method,
                'all_scores': {
                    'ratio': best_matches['ratio'][1],
                    'partial_ratio': best_matches['partial_ratio'][1], 
                    'token_sort_ratio': best_matches['token_sort_ratio'][1],
                    'token_set_ratio': best_matches['token_set_ratio'][1]
                }
            })
        else:
            results.append({
                'new_query_id': new_queries.iloc[i]['id'],
                'new_query': new_queries.iloc[i]['query'],
                'matched_resolved_id': None,
                'matched_resolved_query': 'No match found',
                'expected_match_id': expected_match_id,
                'is_correct_match': False,
                'similarity_score': best_score,
                'best_method': best_method,
                'all_scores': {
                    'ratio': best_matches['ratio'][1],
                    'partial_ratio': best_matches['partial_ratio'][1], 
                    'token_sort_ratio': best_matches['token_sort_ratio'][1],
                    'token_set_ratio': best_matches['token_set_ratio'][1]
                }
            })
    
    return pd.DataFrame(results)

def bow_tfidf_matching(resolved_queries, new_queries, method='tfidf', threshold=0.5):
    """
    Match queries using Bag of Words or TF-IDF with cosine similarity
    """
    # Preprocess queries
    resolved_processed = [preprocess_text(q) for q in resolved_queries['query']]
    new_processed = [preprocess_text(q) for q in new_queries['query']]
    
    # Combine all queries for vectorization
    all_queries = resolved_processed + new_processed
    
    # Choose vectorizer
    if method == 'bow':
        vectorizer = CountVectorizer(stop_words='english', ngram_range=(1, 2))
    else:  # tfidf
        vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
    
    # Fit and transform all queries
    query_vectors = vectorizer.fit_transform(all_queries)
    
    # Split back into resolved and new vectors
    resolved_vectors = query_vectors[:len(resolved_processed)]
    new_vectors = query_vectors[len(resolved_processed):]
    
    # Calculate cosine similarity
    similarity_matrix = cosine_similarity(new_vectors, resolved_vectors)
    
    results = []
    for i, new_query in enumerate(new_queries['query']):
        # Find best match
        best_match_idx = np.argmax(similarity_matrix[i])
        best_score = similarity_matrix[i][best_match_idx]
        
        # Check if we have expected match for comparison
        expected_match_id = new_queries.iloc[i].get('expected_match_id', None)
        is_correct_match = (resolved_queries.iloc[best_match_idx]['id'] == expected_match_id) if expected_match_id else "Unknown"
        
        if best_score >= threshold:
            results.append({
                'new_query_id': new_queries.iloc[i]['id'],
                'new_query': new_query,
                'matched_resolved_id': resolved_queries.iloc[best_match_idx]['id'],
                'matched_resolved_query': resolved_queries.iloc[best_match_idx]['query'],
                'expected_match_id': expected_match_id,
                'is_correct_match': is_correct_match,
                'similarity_score': best_score,
                'method': method
            })
        else:
            results.append({
                'new_query_id': new_queries.iloc[i]['id'],
                'new_query': new_query,
                'matched_resolved_id': None,
                'matched_resolved_query': 'No match found',
                'expected_match_id': expected_match_id,
                'is_correct_match': False,
                'similarity_score': best_score,
                'method': method
            })
    
    return pd.DataFrame(results)

def find_optimal_threshold(resolved_queries, new_queries):
    """
    Find optimal similarity threshold by testing different values
    """
    thresholds = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    
    print("Testing different thresholds for fuzzy matching:")
    for threshold in thresholds:
        fuzzy_results = fuzzy_search_matching(resolved_queries, new_queries, threshold)
        matches = len(fuzzy_results[fuzzy_results['matched_resolved_id'].notna()])
        
        # Calculate accuracy if we have expected matches
        if 'expected_match_id' in new_queries.columns:
            correct_matches = len(fuzzy_results[fuzzy_results['is_correct_match'] == True])
            accuracy = correct_matches / len(new_queries) * 100
            print(f"Threshold {threshold}: {matches}/{len(new_queries)} matches found, Accuracy: {accuracy:.1f}%")
        else:
            print(f"Threshold {threshold}: {matches}/{len(new_queries)} matches found")
    
    print("\nTesting different thresholds for TF-IDF matching:")
    for threshold in [0.1, 0.2, 0.3, 0.4, 0.5]:
        tfidf_results = bow_tfidf_matching(resolved_queries, new_queries, 'tfidf', threshold)
        matches = len(tfidf_results[tfidf_results['matched_resolved_id'].notna()])
        
        # Calculate accuracy if we have expected matches
        if 'expected_match_id' in new_queries.columns:
            correct_matches = len(tfidf_results[tfidf_results['is_correct_match'] == True])
            accuracy = correct_matches / len(new_queries) * 100
            print(f"Threshold {threshold}: {matches}/{len(new_queries)} matches found, Accuracy: {accuracy:.1f}%")
        else:
            print(f"Threshold {threshold}: {matches}/{len(new_queries)} matches found")

In [13]:
# Load data
resolved_df, new_df = load_query_data()
print(f"\nLoaded {len(resolved_df)} resolved queries and {len(new_df)} new queries")

# Find optimal thresholds
find_optimal_threshold(resolved_df, new_df)

# Perform fuzzy matching with optimal threshold
print("\n" + "="*50)
print("FUZZY MATCHING RESULTS (threshold=50)")
print("="*50)
fuzzy_results = fuzzy_search_matching(resolved_df, new_df, threshold=50)
print(fuzzy_results[['new_query_id', 'new_query', 'matched_resolved_id', 'matched_resolved_query', 
                   'expected_match_id', 'is_correct_match', 'similarity_score', 'best_method']])

# Show detailed fuzzy scores for first few examples
print("\nDetailed fuzzy scores for first 3 queries:")
for i in range(min(3, len(fuzzy_results))):
    row = fuzzy_results.iloc[i]
    print(f"\nQuery {i+1}: '{row['new_query']}'")
    print(f"Best match: '{row['matched_resolved_query']}'")
    print(f"All fuzzy scores: {row['all_scores']}")
    print(f"Best method: {row['best_method']}")

# Perform TF-IDF matching
print("\n" + "="*50)
print("TF-IDF MATCHING RESULTS (threshold=0.2)")
print("="*50)
tfidf_results = bow_tfidf_matching(resolved_df, new_df, 'tfidf', threshold=0.1)
print(tfidf_results[['new_query_id', 'new_query', 'matched_resolved_id', 'matched_resolved_query',
                    'expected_match_id', 'is_correct_match', 'similarity_score', 'method']])

# Perform BoW matching
print("\n" + "="*50)
print("BAG OF WORDS MATCHING RESULTS (threshold=0.1)")
print("="*50)
bow_results = bow_tfidf_matching(resolved_df, new_df, 'bow', threshold=0.1)
print(bow_results[['new_query_id', 'new_query', 'matched_resolved_id', 'matched_resolved_query',
                  'expected_match_id', 'is_correct_match', 'similarity_score', 'method']])

Resolved queries columns: ['Query_ID', 'Pre_Resolved_Query']
Sample resolved queries:
   Query_ID                    Pre_Resolved_Query
0         1     Unable to connect to the internet
1         2        Payment failed during checkout
2         3     App crashes when opening settings
3         4   Forgot password and unable to reset
4         5  Unable to upload files to the server

New queries columns: ['Variation_Query', 'Matches_With_Query_ID']
Sample new queries:
                             Variation_Query  Matches_With_Query_ID
0           Unabel to conect to the internet                      1
1                  Can’t connect to internet                      1
2                        Intenet not working                      1
3               Payment failed while chekout                      2
4  Payment did not go through during chckout                      2

Loaded 5 resolved queries and 20 new queries
Testing different thresholds for fuzzy matching:
Threshold 0.3: 20/20 mat

In [14]:
if 'expected_match_id' in new_df.columns:
    # Calculate accuracy for each method
    fuzzy_accuracy = len(fuzzy_results[fuzzy_results['is_correct_match'] == True]) / len(new_df) * 100
    tfidf_accuracy = len(tfidf_results[tfidf_results['is_correct_match'] == True]) / len(new_df) * 100
    bow_accuracy = len(bow_results[bow_results['is_correct_match'] == True]) / len(new_df) * 100
    
    print(f"Fuzzy Search Accuracy: {fuzzy_accuracy:.1f}%")
    print(f"TF-IDF Accuracy: {tfidf_accuracy:.1f}%")
    print(f"Bag of Words Accuracy: {bow_accuracy:.1f}%")
    
    # Best method recommendation
    best_method = max([('Fuzzy Search', fuzzy_accuracy), ('TF-IDF', tfidf_accuracy), ('Bag of Words', bow_accuracy)], 
                     key=lambda x: x[1])
    print(f"\nBest performing method: {best_method[0]} with {best_method[1]:.1f}% accuracy")

# Method analysis
print(f"\nFuzzy Search - Average similarity score: {fuzzy_results['similarity_score'].mean():.3f}")
print(f"TF-IDF - Average similarity score: {tfidf_results['similarity_score'].mean():.3f}")
print(f"Bag of Words - Average similarity score: {bow_results['similarity_score'].mean():.3f}")

Fuzzy Search Accuracy: 100.0%
TF-IDF Accuracy: 90.0%
Bag of Words Accuracy: 95.0%

Best performing method: Fuzzy Search with 100.0% accuracy

Fuzzy Search - Average similarity score: 77.650
TF-IDF - Average similarity score: 0.301
Bag of Words - Average similarity score: 0.372


# TASK 2 - MATCH NAMES WITH VARIATIONS

In [16]:
def preprocess_name(name):
    """
    Preprocess names for better matching:
    - Remove extra spaces
    - Handle different formats
    - Normalize punctuation
    """
    if pd.isna(name):
        return ""
    
    name = str(name).strip()
    
    # Remove extra punctuation and normalize spaces
    name = re.sub(r'[^\w\s,.-]', '', name)
    name = ' '.join(name.split())
    
    return name

def normalize_name_format(name):
    """
    Convert names to a standard format for better comparison
    Handles "Last, First" vs "First Last" format
    """
    name = preprocess_name(name)
    
    # Check if name is in "Last, First" format
    if ',' in name:
        parts = name.split(',')
        if len(parts) == 2:
            last_name = parts[0].strip()
            first_name = parts[1].strip()
            return f"{first_name} {last_name}".strip()
    
    return name

def load_name_data():
    """
    Load base names and name variations from CSV files
    Files structure:
    - base_names.csv: Base_Name_ID, Base_Name
    - name_variations.csv: Variation, Matches_With_Base_Name
    """
    try:
        # Load base names
        base_names_df = pd.read_csv('text_search_data/base_names.csv')
        print("Base names columns:", base_names_df.columns.tolist())
        print("Sample base names:")
        print(base_names_df.head())
        
        # Load name variations
        variations_df = pd.read_csv('text_search_data/name_variations.csv')
        print("\nName variations columns:", variations_df.columns.tolist())
        print("Sample name variations:")
        print(variations_df.head())
        
        # Extract lists for matching
        base_names_list = base_names_df['Base_Name'].tolist()
        variations_list = variations_df['Variation'].tolist()
        expected_matches = variations_df['Matches_With_Base_Name'].tolist()
        
        return base_names_df, variations_df, base_names_list, variations_list, expected_matches
        
    except FileNotFoundError as e:
        print(f"Name files not found: {e}")
        print("Please ensure files are in the text_search_data folder with correct names:")
        print("- base_names.csv")
        print("- name_variations.csv")

def fuzzy_name_matching_with_ground_truth(base_names_list, variations_list, expected_matches, threshold=85):
    """
    Match name variations with base names using fuzzy search
    Compares results with expected matches for accuracy calculation
    """
    results = []
    
    # Preprocess names
    processed_base_names = [normalize_name_format(name) for name in base_names_list]
    processed_variations = [normalize_name_format(name) for name in variations_list]
    
    for i, variation in enumerate(processed_variations):
        best_matches = []
        
        for j, base_name in enumerate(processed_base_names):
            # Test different fuzzy methods for names
            ratio = fuzz.ratio(variation.lower(), base_name.lower())
            partial_ratio = fuzz.partial_ratio(variation.lower(), base_name.lower())
            token_sort = fuzz.token_sort_ratio(variation.lower(), base_name.lower())
            token_set = fuzz.token_set_ratio(variation.lower(), base_name.lower())
            
            # Use the best score among all methods
            best_score = max(ratio, partial_ratio, token_sort, token_set)
            
            best_matches.append({
                'base_name_index': j,
                'base_name': base_names_list[j],
                'similarity_score': best_score,
                'ratio': ratio,
                'partial_ratio': partial_ratio,
                'token_sort': token_sort,
                'token_set': token_set
            })
        
        # Sort matches by score and get the best one
        best_match = max(best_matches, key=lambda x: x['similarity_score'])
        
        # Check if prediction matches expected result
        expected_match = expected_matches[i]
        is_correct = (best_match['base_name'] == expected_match)
        
        if best_match['similarity_score'] >= threshold:
            results.append({
                'variation_name': variations_list[i],
                'predicted_match': best_match['base_name'],
                'expected_match': expected_match,
                'is_correct': is_correct,
                'similarity_score': best_match['similarity_score'],
                'status': 'Match Found',
                'all_fuzzy_scores': {
                    'ratio': best_match['ratio'],
                    'partial_ratio': best_match['partial_ratio'],
                    'token_sort': best_match['token_sort'],
                    'token_set': best_match['token_set']
                }
            })
        else:
            # No match found above threshold
            results.append({
                'variation_name': variations_list[i],
                'predicted_match': 'No match found',
                'expected_match': expected_match,
                'is_correct': False,
                'similarity_score': best_match['similarity_score'],
                'status': 'No Match (Below Threshold)',
                'all_fuzzy_scores': {
                    'ratio': best_match['ratio'],
                    'partial_ratio': best_match['partial_ratio'],
                    'token_sort': best_match['token_sort'],
                    'token_set': best_match['token_set']
                }
            })
    
    return pd.DataFrame(results)

def test_different_thresholds_names(base_names_list, variations_list, expected_matches):
    """
    Test different similarity thresholds for name matching with accuracy calculation
    """
    thresholds = [60, 70, 80, 85, 90, 95]
    
    print("Testing different thresholds for name matching:")
    for threshold in thresholds:
        results = fuzzy_name_matching_with_ground_truth(base_names_list, variations_list, expected_matches, threshold)
        matches = len(results[results['similarity_score'] >= threshold])
        correct_matches = len(results[results['is_correct'] == True])
        accuracy = correct_matches / len(variations_list) * 100
        print(f"Threshold {threshold}: {matches}/{len(variations_list)} matches found, Accuracy: {accuracy:.1f}%")

In [ ]:
# Load name data
base_names_df, variations_df, base_names_list, variations_list, expected_matches = load_name_data()
print(f"\nLoaded {len(base_names_list)} base names and {len(variations_list)} name variations")

print("\nBase Names:", base_names_list)
print("Name Variations:", variations_list)
print("Expected Matches:", expected_matches)

# Test different thresholds
test_different_thresholds_names(base_names_list, variations_list, expected_matches)

# Perform name matching with optimal threshold
print("\n" + "="*50)
print("NAME MATCHING RESULTS (threshold=85)")
print("="*50)
name_results = fuzzy_name_matching_with_ground_truth(base_names_list, variations_list, expected_matches, threshold=85)
print(name_results[['variation_name', 'predicted_match', 'expected_match', 'is_correct', 'similarity_score', 'status']])

# Show summary statistics
print("\n" + "="*50)
print("SUMMARY STATISTICS")
print("="*50)
correct_matches = name_results[name_results['is_correct'] == True]
total_variations = len(name_results)
accuracy = len(correct_matches) / total_variations * 100

print(f"Total variations processed: {total_variations}")
print(f"Correct matches: {len(correct_matches)}")
print(f"Accuracy: {accuracy:.1f}%")
print(f"Average similarity score: {name_results['similarity_score'].mean():.2f}")
print(f"Minimum similarity score: {name_results['similarity_score'].min():.2f}")
print(f"Maximum similarity score: {name_results['similarity_score'].max():.2f}")

# Analyze specific cases
matches_found = name_results[name_results['status'] == 'Match Found']
no_matches = name_results[name_results['status'] == 'No Match (Below Threshold)']

print(f"\nMatches found above threshold: {len(matches_found)}")
print(f"No matches (below threshold): {len(no_matches)}")

if len(no_matches) > 0:
    print("\nVariations that didn't match (below threshold):")
    for _, row in no_matches.iterrows():
        print(f"  '{row['variation_name']}' -> Expected: '{row['expected_match']}' (Score: {row['similarity_score']:.1f})")

Base names columns: ['Base_Name_ID', 'Base_Name']
Sample base names:
   Base_Name_ID         Base_Name
0             1        John Smith
1             2    Jennifer Brown
2             3  Michael O'Connor
3             4      Maria Garcia
4             5        Robert Lee

Name variations columns: ['Variation', 'Matches_With_Base_Name']
Sample name variations:
      Variation Matches_With_Base_Name
0  Thomas  King            Thomas King
1    ThomasKing            Thomas King
2  Maria Garcia           Maria Garcia
3     MaryLewis             Mary Lewis
4      Nancy W.           Nancy Wright

Loaded 20 base names and 100 name variations

Base Names: ['John Smith', 'Jennifer Brown', "Michael O'Connor", 'Maria Garcia', 'Robert Lee', 'Linda Johnson', 'William Davis', 'Elizabeth Wilson', 'David Martinez', 'Susan Clark', 'James Rodriguez', 'Mary Lewis', 'Paul Allen', 'Karen Young', 'Thomas King', 'Nancy Wright', 'Daniel Scott', 'Sandra Hill', 'Christopher Green', 'Jessica Adams']
Name Variati